In [ ]:
import nltk
from keras.src.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
# import tkinter as tk
# from tkinter import StringVar
import tensorflow as tf
import keras
from nltk.tokenize import word_tokenize

nltk.download('punkt')
# Load your corpus
with open("/home/mukul-barhate/Machine_learning/pizza.txt", 'r') as file:
    text = file.read()
corpus =text
# Tokenize the corpusa
tokens = word_tokenize(corpus.lower())

# Prepare sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for i in range(1, len(tokens)):
    n_gram_sequence = tokens[:i+1]
    encoded_sequence = tokenizer.texts_to_sequences([' '.join(n_gram_sequence)])[0]
    input_sequences.append(encoded_sequence)

# Pad sequences and create predictors and label
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=total_words)

# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=2, verbose=1)

# Define the prediction function
def predict_next_words(model, tokenizer, text, num_words=3):
    for _ in range(num_words):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(sequence), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        text += " " + output_word
    return text

# Tkinter GUI
# def on_button_click():
#     user_input = entry.get()
#     prediction = predict_next_words(model, tokenizer, user_input)
#     prediction_var.set(f"Predicted next words: {prediction}")

# # Initialize the main window
# root = tk.Tk()
# root.title("Next Word Predictor")

# # Create GUI elements
# label = tk.Label(root, text="Enter your text:")
# label.pack()

# entry = tk.Entry(root, width=50)
# entry.pack()

# button = tk.Button(root, text="Predict Next Words", command=on_button_click)
# button.pack()

# prediction_var = StringVar()
# prediction_label = tk.Label(root, textvariable=prediction_var)
# prediction_label.pack()

# # Start the Tkinter event loop
# root.mainloop()

[nltk_data] Downloading package punkt to /home/mukul-
[nltk_data]     barhate/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Epoch 1/2


/home/mukul-barhate/myenv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


31/60 ━━━━━━━━━━━━━━━━━━━━ 1:30 3s/step - accuracy: 0.0345 - loss: 6.3087